## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Jazmin Reyna

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [2]:
df_X, s_y = load_diabetes(return_X_y=True, as_frame = True)

## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [3]:
def get_linear_regression_model( df_X, s_y ):
    # your code here
    s_y = np.array([s_y])
    X = pd.concat([pd.DataFrame({'intercept': np.ones(len(df_X))}), df_X], axis=1)
    beta_hat, residuals, rank, s = np.linalg.lstsq(X,s_y.T,rcond=None)
    return beta_hat
# b = get_linear_regression_model( df_X, s_y )

In [4]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([[ 4.18818425],
       [ 1.77890808],
       [ 0.74032569],
       [-1.3506416 ],
       [ 0.14535984]])

In [5]:
def predict_linear_regression_value( beta_hat, df_X ):
    # your code here
    X = pd.concat([pd.DataFrame({'intercept': np.ones(len(df_X))}), df_X], axis=1)
    return np.matmul(X, beta_hat)


In [6]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

,0
0,4.036695
1,4.395026
2,3.908847


## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [7]:
from sklearn.utils import shuffle

def partition_data( df_X, s_y, k ):
    # your code here
    df_X, s_y = shuffle(df_X, s_y)
    # caused error -> df_x_random = df_X.sample(frac = 1)
    dict_k_df_X, dict_k_s_y = {}, {}
    
    for i in range(k):
        if i == k-1:
            dict_k_df_X[i+1] = df_X.iloc[i*(len(df_X)//k):len(df_X)]
            dict_k_s_y[i+1] = s_y.iloc[i*(len(df_X)//k):len(df_X)]
        else:
            dict_k_df_X[i+1] = df_X.iloc[i*(len(df_X)//k):(i+1)*(len(df_X)//k)]
            dict_k_s_y[i+1] = s_y.iloc[i*(len(df_X)//k):(i+1)*(len(df_X)//k)]
    
    return dict_k_df_X, dict_k_s_y

In [8]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [9]:
# Check fold sizes
total = 0
for i, j in dict_k_df_X.items():
    print('Fold', i, 'length of dataframe is', len(j), 'and length of series is', len(dict_k_s_y[i]))
    total += len(dict_k_df_X[i])
print('The sum of the number of elements in each fold is', total, 'and there are', len(df_X), 'rows in the original df')

Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 88 and length of series is 88
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 88 and length of series is 88
Fold 5 length of dataframe is 90 and length of series is 90
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [10]:
def get_mae( s_y, s_y_hat):
    # your code here
    mae = 0
    
    size = len(s_y)
    for i in range(size):
        err = abs((s_y[i] - s_y_hat[i])/size)
        mae += err
    return mae

In [11]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [12]:
mae = np.array([])
# print(dict_k_df_X.items())
for k in dict_k_df_X.keys():
    # your code here
    test_X = dict_k_df_X[k] 
    test_s_y = dict_k_s_y[k]
    
    train_X = pd.DataFrame()
    train_s_y = pd.Series(dtype = float)
    
    for i in dict_k_df_X.keys():
        if i != k:
            train_X = pd.concat([train_X, dict_k_df_X[i]], axis = 0)
            train_X = train_X.reset_index(drop = True)
            train_s_y = pd.concat([train_s_y, dict_k_s_y[i]], axis = 0)
            train_s_y = train_s_y.reset_index(drop = True)
    
    beta_hat_temp = get_linear_regression_model(train_X, train_s_y)
#     print(beta_hat_temp)
    s_y_hat = predict_linear_regression_value(beta_hat_temp, test_X.reset_index(drop = True))
#     print(s_y_hat)
    
    mae = np.append( mae, get_mae(test_s_y.values, s_y_hat.values) ) 
#     print('fold', k, 'mae =' , get_mae(test_s_y.values, s_y_hat.values))
mae


array([41.22979239, 43.48725806, 45.66187803, 50.31019762, 42.96187417])

In [13]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 41.23, the max MAE is 50.31, and the mean MAE is 44.73


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [14]:
df_X, s_y = load_iris(return_X_y=True, as_frame = True)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [15]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [16]:
def get_acc( s_1, s_2 ):
    # your code here
    return np.sum(s_1 == s_2)/len(s_1)


In [17]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [18]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])
# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    print('Outer fold', k,'out of 5')
    
    avg_accs = [] 
    for pos_min_impurity in possible_min_impurity_decrease:
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k
        print('Testing' ,pos_min_impurity, 'min impurity decrease')
        
        total_acc = 0
        for j in dict_k_df_X.keys():
            if j != k:
                inner_train_X = pd.DataFrame()
                inner_train_y = pd.Series(dtype = float)
                for i in dict_k_df_X.keys():
                    if i != k and i != j:
                        inner_train_X = pd.concat([inner_train_X, dict_k_df_X[i]], axis = 0)
                        inner_train_X = inner_train_X.reset_index(drop = True)
                        
                        inner_train_y = pd.concat([inner_train_y, dict_k_s_y[i]], axis = 0)
                        inner_train_y = inner_train_y.reset_index(drop = True)
                
                inner_test_X = dict_k_df_X[j].reset_index(drop = True)
                inner_test_y = dict_k_s_y[j].reset_index(drop = True)

                clf = tree.DecisionTreeClassifier(min_impurity_decrease = pos_min_impurity, criterion = 'gini')
                clf = clf.fit(inner_train_X, inner_train_y)
                
                this_acc = get_acc(inner_test_y, clf.predict(inner_test_X))
                
                total_acc += this_acc
        
        avg_acc = total_acc/4
        print('    Average accuracy over 4 folds is', avg_acc)
        avg_accs.append(avg_acc)
        
    # Use best min impurity decrease to train model    
    train_x = pd.DataFrame()
    train_y = pd.Series(dtype = float)
    
    for i in dict_k_df_X.keys():
        if i != k:
            train_x = pd.concat([train_x, dict_k_df_X[i]], axis = 0)
            train_x = train_x.reset_index(drop = True)
            
            train_y = pd.concat([train_y, dict_k_s_y[i]], axis = 0)
            train_y = train_y.reset_index(drop = True)
    
    
    # Find the min_impurity_decrease that had the best accuracy
    best_acc = max(avg_accs)
    idx_of_highest_acc = avg_accs.index(best_acc)
    best_min_imp = possible_min_impurity_decrease[idx_of_highest_acc]
    
    clf = tree.DecisionTreeClassifier(min_impurity_decrease = best_min_imp, criterion = 'gini')
    clf = clf.fit(train_x, train_y)
    
    test_x = dict_k_df_X[k].reset_index(drop = True)
    test_y = dict_k_s_y[k].reset_index(drop = True)
    
    print('')
    print('Best min impurity decrease is', best_min_imp)
    print('')
    
    this_acc = get_acc(test_y, clf.predict(test_x))   
      
    # outer accuracy calculation 
    outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop


Outer fold 1 out of 5
Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9416666666666667
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.9416666666666667
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.75
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.3416666666666666

Best min impurity decrease is 0.1

Outer fold 2 out of 5
Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9166666666666667
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.8416666666666667
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.6666666666666666
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.2833333333333333

Best min impurity decrease is 0.1

Outer fold 3 out of 5
Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.8666

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [19]:
print("The min accuracy is {:.2f}, the max accuracy is {:.2f}, and the mean accuracy is {:.2f}".format(outer_acc.min(),outer_acc.max(),outer_acc.mean()))

The min accuracy is 0.87, the max accuracy is 1.00, and the mean accuracy is 0.94


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=828f9502-5e28-449c-93f9-65ac8effe6b6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>